In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [17]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
PostCode = df["Postcode"].tolist()
Borough = df["Borough"].tolist()
Neighbourhood=df["Neighbourhood"].tolist()
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
df=df[df.Borough != 'Not assigned']

In [24]:
df['Postcode'] = df['Postcode'].astype(str)
df['Neighbourhood'] = df['Neighbourhood'].astype(str)
result = df.groupby(['Postcode','Borough'], sort=False).agg( ','.join)
result.head()



,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned


In [25]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

In [27]:
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


## I cleaned dataframe and dropped unnecessary values, grouped by Postcode, so size is much smaller.

In [28]:
df.shape

(211, 3)